In [1]:
import pandas as pd
import numpy as np
import sklearn.ensemble

#importing RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier 

#get_ipython().magic('matplotlib inline')
import matplotlib.pyplot as py

In [ ]:
#reading the data file
raw_file = pd.read_csv('ceu_yri_chb_A.raw', sep=" ",nrows=310)
raw_file.head(3)

In [ ]:
#deleting the columns not part of the model
raw_file = raw_file.drop(raw_file.columns[[0,1,2,3,4,5]],axis=1)
#print(raw_file.head())

In [ ]:
snps = list(raw_file.columns.values)

In [ ]:
len(snps)

In [ ]:
pops = pd.read_csv('pop_names', sep=" ",nrows=310)
pops = pops.drop('IID',axis=1)


In [ ]:
y = np.array(pops['pop'])
#converting the covariates into a matrix
X = raw_file.as_matrix(columns=None)


In [ ]:
num_estimators = [250,500,1000,2000,3000,4000,5000]
oob_scores = []

In [ ]:
#loop to create the models with different estimator counts
for num in num_estimators:
    rf = RandomForestClassifier(n_estimators=num,criterion='gini',oob_score=True)
    rf.fit(X,y)
    oob_scores.append(rf.oob_score_)
    importances = rf.feature_importances_
    importances_dict, sorted_importances_dict = {},{}
    #filtering importances
    for i in range(len(importances)):
        if(importances[i]>0):
            importances_dict[snps[i]] = importances[i]
    #sorting importance dictionary
    sorted_importances_snps = sorted(importances_dict, key=lambda x: importances_dict[x], reverse = True)
    #generating sorted importances dictionary
    for snp in sorted_importances_snps:
        sorted_importances_dict[snp] = importances_dict[snp]
    #witing to a file
    fw = open("Importances_rf//importances_"+str(num)+".txt","w")
    fw.write("SNP\tImportance\n")
    for key,val in sorted_importances_dict:
        fw.write(str(key)+'\t')
        fw.write(str(val)+'\n')
    fw.close()

In [ ]:
oob_scores = np.array(oob_scores)
oob_errors = 1 - oob_scores

In [ ]:
#print(oob_errors)
#plotting the oob error vs number of trees in rf
py.plot(num_estimators,oob_errors)
py.xlabel('Number of Trees')
py.ylabel('OOB Error')
py.show()